In [1]:
import pandas as pd
import data_util
from tqdm.notebook import tqdm
#from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from tec_an import *
import numpy as np
from data_util import *
from sklearn_model_hyper import *
from joblib import dump, load
from data_generator import *
from data_agent import *
from stock_agent import *
from backtest import *
from bitstamp import *

from sklearn.utils import shuffle

import math
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as scs
import scikitplot as skplt

from keras.wrappers.scikit_learn import KerasClassifier
from imblearn.over_sampling import RandomOverSampler
import random
from catboost import CatBoost

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.utils.validation import check_is_fitted

from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import *
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE, RFECV

path = "./data/"


In [2]:
#train_start_list = [1622502000, 1590966000, 1559343600, 1580515200, 1612137600]
train_start_list = [1622502000, 1590966000, 1559343600, 1527807600, 1517443200, 1548979200, 1580515200, 1612137600]
#train_start_list = [1612137600]
train_keys = ["btcusd", "ethusd", "bchbtc"]
#train_keys = ["bchbtc"]
val_start = 1626340500
val_end = 1626369600

In [3]:
load_from_disk = True

# '2Min'

In [4]:
def get_online_data(minutes, source_data_generator, load_from_disk, file_prefix = ""):
    
    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = train_keys,
                 train_limit = 1000,
                 val_limit = 1000,
                 val_keys = ["btcusd"],
                 val_start = val_start,
                 val_end = val_end,
                 train_start_list = train_start_list
    )

    online_path = f'data/online{file_prefix}_{minutes}'
    
    if (load_from_disk):
        online = load(online_path)    
    else:
        #online.load_train_cache()
        online.load_cache()
        online.sourceDataGenerator = None
        dump(online, online_path)
        
    
    return online

In [5]:

def getModel():
    param = {'criterion': 'entropy', 'max_depth': 100, 'n_estimators': 30, 'random_state': 42}
    #param = {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 3, 'random_state': 42}
    coModel = RandomForestClassifier(**param)
    return coModel

In [16]:
%%time

def get_classifiers():
    clss = classifiers = [
        #lambda : MLPClassifier(alpha=1,
        #                       solver = 'lbfgs', 
        #                       max_iter=1000000, 
        #                       random_state = 42),
        #lambda : MLPClassifier(alpha=1, max_iter=10000, random_state = 42),
        
        
        lambda : make_pipeline( StandardScaler(),
        MLPClassifier(
            solver='lbfgs', alpha=1, random_state=1, max_iter=2000,
            early_stopping=True)
        ),
        
        lambda : xgb.XGBClassifier(random_state=1,
                                   objective = "binary:logistic",
                                   eval_metric='mlogloss',
                                   learning_rate=0.01, 
                                   use_label_encoder = True),
        
        #lambda : GaussianNB(),
        #lambda : QuadraticDiscriminantAnalysis(),
        #lambda : AdaBoostClassifier(random_state = 42),
        #lambda : CatBoostClassifier(logging_level = 'Silent'),
        
        
        #lambda : LogisticRegression(),
        #lambda : KNeighborsClassifier(3),
        #lambda : GaussianProcessClassifier(1.0 * RBF(1.0)),
        
        
        #lambda : DecisionTreeClassifier(random_state = 42),
        #lambda : RandomForestClassifier(max_depth=50, n_estimators=100, max_features=1),
        
        #lambda : SVC(gamma=2, C=1, random_state = 42),
    ]
    
    def rand():
        ##print("rand created")
        param = {'criterion': 'entropy', 'max_depth': 100, 'n_estimators': 30, 'random_state': 42}
        return RandomForestClassifier(**param)
    

    #clss = []
    clss.append(lambda : rand())
    return clss


estimators = [
                #RFE(estimator=DecisionTreeClassifier(random_state=10), n_features_to_select=6),
                lambda : RFE(estimator=RandomForestClassifier(random_state=10), n_features_to_select=10),
                lambda : RFE(estimator=DecisionTreeClassifier(random_state=10), n_features_to_select=8),
                lambda : RFECV(estimator=DecisionTreeClassifier(random_state=10)),
                lambda : RFECV(estimator=RandomForestClassifier(random_state=10))
                ]


def get_all_models():
    models = []
    classifiers = get_classifiers()
    for cls in classifiers:
        models.append(cls())
        #models.append(MockModel(cls()))
        #models.append(MockCoPilotModel(cls(), getModel()))
        
        for est in estimators:
            tmp = lambda : Pipeline(
                steps=[
                ('s',est()),
                ('m',cls())
            ])
            models.append(tmp())
            #models.append(MockModel(tmp()))
            #models.append(MockCoPilotModel(tmp(), getModel()))
    return models


CPU times: user 10 µs, sys: 1 µs, total: 11 µs
Wall time: 13.8 µs


In [17]:
get_all_models_factory = lambda : get_all_models()

In [18]:
%%time

def test_models(provider, get_all_models_factory, steps = [1]):
    
    score_board_by_step = {}

    for step in steps:
        print(f"Step {step}")
        models_list = get_all_models_factory()
        model_result = []
        for idx in tqdm(range(len(models_list))):
            model_creator = models_list[idx]
            model = model_creator
            #print(f"Trainning {model}")
            result = run_trial(model_creator, provider, step)

            result['step'] = step

            model_result.append(result)
        
        score_board_by_step[step] = model_result
        
    best_models = []

    best_model = []

    best_score = -1
    
    best_step = -1
    
    model_rank = []
    
    def myFunc(e):
        return e['profit']

    for step_idx in score_board_by_step:
        step_board = score_board_by_step[step_idx]
        for result in step_board:
            if (result['profit'] > 0):
                model_rank.append(result)
            
    model_rank.sort(key=myFunc, reverse = True)
    
    return model_rank

CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 11 µs


In [19]:

steps = [1, 5]

windows = [5, 10, 20, 30, 40]


In [20]:

def test_windows(minutes, steps = steps):
    results = []
    for win in tqdm(windows):
        tec = TecAn(windows = win, windows_limit = 100)
        sourceDataGenerator = SourceDataGenerator(tec = tec)
        online = get_online_data(minutes, sourceDataGenerator, load_from_disk, win)
        result = test_models(
            online, 
            get_all_models_factory,
            steps = steps
        )
        for rank in result:
            rank['window'] = win
            rank['minutes'] = minutes
            
        results.append(result)
    return results
    

In [21]:
%%time

minutes = [15, 5, 3, 1]

min_results = []

for minu in minutes:
    print(f"Evaluating {minu} min")
    results = test_windows(minu, steps)
    min_results.append(results)


Evaluating 15 min


  0%|          | 0/5 [00:00<?, ?it/s]

Step 1


  0%|          | 0/15 [00:00<?, ?it/s]

/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.

...

/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.

[23:54:19] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:54:44] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:54:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameter

  0%|          | 0/15 [00:00<?, ?it/s]

/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.

[00:08:27] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:08:51] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:08:54] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameter

['model/results']

In [137]:
best_results = []
for results in min_results:
    for r in results:
        if (len(r) > 0):
            for win_result in r:
                profit = win_result['profit']
                model = win_result['model']
                window = win_result['window']
                minutes = win_result['minutes']
                step = win_result['step']
                profit = win_result['profit']
                if (profit > 101 and minutes == 15):
                    print(f"Minutes={minutes} Window={window} Step={step} | {profit}")
                    print(f"{win_result['model']}")
                    print(f"")
                    best_results.append(win_result)
        else:
            print("Not found")

print(f"{len(best_results)} selected")

Minutes=15 Window=5 Step=1 | 104.68849807238843
Pipeline(steps=[('s',
                 RFE(estimator=RandomForestClassifier(random_state=10),
                     n_features_to_select=10)),
                ('m',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, eval_metric='mlogloss',
                               gamma=0, gpu_id=-1, importance_type='gain',
                               interaction_constraints='', learning_rate=0.01,
                               max_delta_step=0, max_depth=6,
                               min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=0, num_parallel_tree=1, random_state=1,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               subsample=1, tree_method='exact',

Pipeline(steps=[('s',
                 RFE(estimator=RandomForestClassifier(random_state=10),
                     n_features_to_select=10)),
                ('m',
                 Pipeline(steps=[('standardscaler', StandardScaler()),
                                 ('mlpclassifier',
                                  MLPClassifier(alpha=1, early_stopping=True,
                                                max_iter=2000, random_state=1,
                                                solver='lbfgs'))]))])

Minutes=15 Window=40 Step=1 | 102.15028907102514
Pipeline(steps=[('s',
                 RFE(estimator=DecisionTreeClassifier(random_state=10),
                     n_features_to_select=8)),
                ('m',
                 Pipeline(steps=[('standardscaler', StandardScaler()),
                                 ('mlpclassifier',
                                  MLPClassifier(alpha=1, early_stopping=True,
                                                max_iter=2000, random_stat

In [128]:
model_path = f'model/results'
dump(best_results, model_path)

['model/results']

In [138]:
def myFunc(e):
    return e['profit']

best_results.sort(key=myFunc, reverse = False)

for best in best_results:
    model = best['model']
    window = best['window']
    minutes = best['minutes']
    step = best['step']
    profit = best['profit']
    print(f"Minutes={minutes} Window={window} Step={step} | {profit}")
    print(f"{model}")
    model_path = f'model/result_{window}_{minutes}_{step}'
    dump(best, model_path)
    print(f"")

Minutes=15 Window=20 Step=1 | 101.12714647457791
Pipeline(steps=[('s', RFECV(estimator=RandomForestClassifier(random_state=10))),
                ('m',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, eval_metric='mlogloss',
                               gamma=0, gpu_id=-1, importance_type='gain',
                               interaction_constraints='', learning_rate=0.01,
                               max_delta_step=0, max_depth=6,
                               min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=0, num_parallel_tree=1, random_state=1,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               subsample=1, tree_method='exact',
                               use_label_encoder=True, vali


Minutes=15 Window=40 Step=1 | 103.34258091369581
Pipeline(steps=[('s',
                 RFE(estimator=RandomForestClassifier(random_state=10),
                     n_features_to_select=10)),
                ('m',
                 Pipeline(steps=[('standardscaler', StandardScaler()),
                                 ('mlpclassifier',
                                  MLPClassifier(alpha=1, early_stopping=True,
                                                max_iter=2000, random_state=1,
                                                solver='lbfgs'))]))])

Minutes=15 Window=40 Step=1 | 103.43532307816722
Pipeline(steps=[('s', RFECV(estimator=DecisionTreeClassifier(random_state=10))),
                ('m',
                 Pipeline(steps=[('standardscaler', StandardScaler()),
                                 ('mlpclassifier',
                                  MLPClassifier(alpha=1, early_stopping=True,
                                                max_iter=2000, random_state=1,
     

In [105]:
def myFunc(e):
    return e['profit']

best_results.sort(key=myFunc, reverse = True)



{'profit': 110.57540259137214,
 'model': Pipeline(steps=[('s', RFECV(estimator=DecisionTreeClassifier(random_state=10))),
                 ('m',
                  XGBClassifier(base_score=0.5, booster='gbtree',
                                colsample_bylevel=1, colsample_bynode=1,
                                colsample_bytree=1, eval_metric='mlogloss',
                                gamma=0, gpu_id=-1, importance_type='gain',
                                interaction_constraints='', learning_rate=0.01,
                                max_delta_step=0, max_depth=6,
                                min_child_weight=1, missing=nan,
                                monotone_constraints='()', n_estimators=100,
                                n_jobs=0, num_parallel_tree=1, random_state=1,
                                reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                                subsample=1, tree_method='exact',
                                use_label_encoder=True,

In [109]:
best_results[3]

{'profit': 105.88279599087541,
 'model': Pipeline(steps=[('s',
                  RFE(estimator=DecisionTreeClassifier(random_state=10),
                      n_features_to_select=8)),
                 ('m',
                  XGBClassifier(base_score=0.5, booster='gbtree',
                                colsample_bylevel=1, colsample_bynode=1,
                                colsample_bytree=1, eval_metric='mlogloss',
                                gamma=0, gpu_id=-1, importance_type='gain',
                                interaction_constraints='', learning_rate=0.01,
                                max_delta_step=0, max_depth=6,
                                min_child_weight=1, missing=nan,
                                monotone_constraints='()', n_estimators=100,
                                n_jobs=0, num_parallel_tree=1, random_state=1,
                                reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                                subsample=1, tree_method='e

In [77]:
def load_online(minutes, window, val_end = 1626388860, currency = "btcusd"):
    tec = TecAn(windows = window, windows_limit = 100)
    source_data_generator = SourceDataGenerator(tec = tec)


    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = ["btcusd"],
                 train_limit = 40,
                 val_limit = 999,
                 val_keys = [currency],
                 #val_start = val_start,
                 val_end = val_end,
                 train_start_list = [1626386301]
    )

    online.load_cache()
    return online

In [78]:
def backtest_model(model, x, closed_prices, back):
    
    for idx in range(len(x)):
        xx = [x[idx]]
        yy = model.predict(xx)[0]
        price = closed_prices[idx]
        #print(f'{idx} {yy} {price}')
        back.on_state(0, price)
        if(yy == 1):
            back.request_buy(price)
        else:
            back.request_sell(price)
    
    print(f"Closing backtest_model {price} {back.current}")
    back.sell(price)
    print(f"Closed backtest_model {back.current}")
    #back.request_sell(31744.62)
    
    return back

In [79]:
def test_model(model, set_key, provider, step, verbose = True):
    valX, valY = provider.load_val_data(set_key)

    x, y, closed_prices = get_sequencial_data(valX, valY, step)
    
    #print(len(x))

    preds = model.predict(x)

    recall = recall_score(y, preds)
    precision = precision_score(y, preds)
    f1 = f1_score(y, preds)
    accuracy = accuracy_score(y, preds)

    back = BackTest(value = 100, 
                    verbose = verbose, 
                    sell_on_profit = True,
                    pending_sell_steps = step)
    back = backtest_model(model, x, closed_prices, back)
    return back

In [123]:
timestamp = int(datetime.timestamp((datetime.now())))


#timestamp = 1610959352

In [124]:
backs = []
fails = []

currency = "btcusd"

online_cache = {}

for best in best_results:
    model = best['model']
    window = best['window']
    minutes = best['minutes']
    step = best['step']
    profit = best['profit']
    print(f"Minutes={minutes} Window={window} Step={step} | {profit}")
    cache_key = f"{minutes}-{window}"
    try:
        online = online_cache[cache_key]
    except :
        print(f"Not found {cache_key}")
        online = load_online(minutes = minutes, window = window, val_end = timestamp, currency = currency)
        online_cache[cache_key] = online
        
    try:
        back = test_model(model, currency, online, step, False)
        backs.append(back)
    except :
        fails.append(best)


Minutes=15 Window=40 Step=1 | 110.57540259137214
Not found 15-40
https://www.bitstamp.net/api/v2/ohlc/btcusd/ -> {'currency_pair': 'btcusd', 'step': 900, 'limit': 40, 'start': 1626386301}
Downloaded from 2021-07-15 22:45:00 to 2021-07-16 08:30:00 31673.68
Resample 15Min - TecAn ( windows 40, windows_limit 100 )
Processing 39 of btcusd


  0%|          | 0/39 [00:00<?, ?it/s]

https://www.bitstamp.net/api/v2/ohlc/btcusd/ -> {'currency_pair': 'btcusd', 'step': 900, 'limit': 999, 'start': 1623435037, 'end': 1626602605}
Downloaded from 2021-07-08 01:30:00 to 2021-07-18 11:00:00 31755.48
Resample 15Min - TecAn ( windows 40, windows_limit 100 )
Processing 998 of btcusd


  0%|          | 0/998 [00:00<?, ?it/s]

Closing backtest_model 31755.48 11.17842080668656
Closed backtest_model 111.17842080668656
Minutes=15 Window=20 Step=1 | 110.257327069269
Not found 15-20
https://www.bitstamp.net/api/v2/ohlc/btcusd/ -> {'currency_pair': 'btcusd', 'step': 900, 'limit': 40, 'start': 1626386301}
Downloaded from 2021-07-15 22:45:00 to 2021-07-16 08:30:00 31673.68
Resample 15Min - TecAn ( windows 20, windows_limit 100 )
Processing 39 of btcusd


...

Closing backtest_model 31700.0 2.084652937031109
Closed backtest_model 101.71376236279345
Minutes=15 Window=5 Step=1 | 101.30572370010182
Closing backtest_model 31755.48 2.787619914474888
Closed backtest_model 101.80028324158683
Minutes=15 Window=30 Step=1 | 101.29457091820967
Closing backtest_model 31755.48 -4.093221932820285
Closed backtest_model 95.90677806717972
Minutes=3 Window=5 Step=1 | 101.28497898436324
Closing backtest_model 31700.0 -0.23596915921035588
Closed backtest_model 99.5882520386677
Minutes=3 Window=10 Step=1 | 101.24857800526418
Closing backtest_model 31700.0 2.0334895850382964
Closed backtest_model 101.78276333480545
Minutes=15 Window=40 Step=1 | 101.17250239487848
Closing backtest_model 31755.48 5.384296754892034
Closed backtest_model 104.39696008200397
Minutes=5 Window=40 Step=1 | 101.15555838673109
Closing backtest_model 31755.48 -4.7518145388341395
Closed backtest_model 95.14522275895645
Minutes=15 Window=30 Step=1 | 101.12761903530297
Closing backtest_model 31

  0%|          | 0/39 [00:00<?, ?it/s]

https://www.bitstamp.net/api/v2/ohlc/btcusd/ -> {'currency_pair': 'btcusd', 'step': 60, 'limit': 999, 'start': 1623435037, 'end': 1626602605}
Downloaded from 2021-07-17 18:25:00 to 2021-07-18 11:03:00 31700.00
Resample 1Min - TecAn ( windows 5, windows_limit 100 )
Processing 998 of btcusd


  0%|          | 0/998 [00:00<?, ?it/s]

Closing backtest_model 31700.0 1.4170784939159091
Closed backtest_model 101.29498798940644
Minutes=3 Window=10 Step=1 | 101.03981296091241
Minutes=3 Window=10 Step=1 | 101.03981296091241
Closing backtest_model 31700.0 3.8660778065078034
Closed backtest_model 103.69029900438586


In [125]:
profs = []
current = []
for back in backs:
    profs.append(back.get_profit())
    current.append(back.current)
    print(f"{back.get_profit()}  {back.current}")

    
print()
print(f"{np.average(profs)}  {np.average(current)}")

11.1784  111.17842080668656
8.9928  108.99278233692455
1.9866  101.98657480066932
4.8488  104.84877635326086
1.9396  101.93961591577452
1.8881  101.88806954332966
1.5114  101.51140921982487
4.6932  104.69324414993085
0.3998  100.39979754223121
6.2889  106.28886666045754
0.4393  100.43926236804795
1.8169  101.81691076459566
1.102  101.10201189719115
0.1919  100.19190137082107
2.7828  102.78275848216926
2.7828  102.78275848216926
3.3716  103.37161654319402
3.5839  103.58387350610231
0.4289  100.42888680926772
2.6824  102.68242969758798
4.0776  104.07759558276604
-1.7179  98.2820510731635
-1.15  98.849952771787
4.7956  104.79555826433585
3.5428  103.54280879045122
0.4226  100.42258897800441
2.449  102.44899105665436
1.9927  101.9927201812034
3.5659  103.56589764746202
1.5043  101.50425623539824
-0.5615  99.43848269855486
3.9127  103.9127255126473
2.3257  102.32574410378834
2.3257  102.32574410378834
2.4694  102.46938976426335
-0.7153  99.2847289702467
-0.7153  99.2847289702467
-1.5146  98